In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import re
from zipfile import ZipFile
from shapely.geometry import Point
from geopy.distance import distance
from geopy import Point as GeoPoint
from fuzzywuzzy import fuzz, process

# Sample FAA UAS incident string
sample_string = """PRELIM INFO FROM FAA OPS: HARRISBURG, PA/UAS INCIDENT/1057E/CIRRUS SR20, REPORTED A COPPER UAS WHILE E BOUND AT 5,000 FEET 5 W HARRISBURG. NO EVASIVE ACTION REPORTED. PA STATE PD NOTIFIED."""
print("sample_string:", sample_string)

# === Load airport data ===
with ZipFile("all-airport-data.xlsx.zip") as z:
    with z.open("all-airport-data.xlsx") as f:
        airport_codes_2 = pd.read_excel(f)

airport_trimmed = airport_codes_2.rename(columns={
    "Loc Id": "faa_code",
    "ARP Latitude DD": "latitude",
    "ARP Longitude DD": "longitude",
    "Name": "Facility Name"
}).dropna(subset=["latitude", "longitude"])

# === Extract facility name from summary ===
def extract_city_state(summary):
    match = re.search(r'FROM FAA OPS:\s+([A-Z\s]+),\s+([A-Z]{2})', summary.upper())
    return f"{match.group(1).title()} {match.group(2)}" if match else None


facility = extract_city_state(sample_string)
print("Extracted facility:", facility)

# === Unified facility matcher (always filters by state + real airports only) ===
facility = extract_city_state(sample_string)
print("Extracted facility:", facility)

if facility is not None:
    try:
        city_part, state_part = facility.rsplit(" ", 1)

        # Filter to real airports only in correct state
        filtered_airports = airport_trimmed[
            (airport_trimmed["State Id"].str.upper() == state_part.upper()) &
            (airport_trimmed["Facility Type"].str.upper().str.contains("AIRPORT"))
        ]
        print(f"Filtered airports in {state_part.upper()}: {len(filtered_airports)}")

        # Fuzzy match city to real airport name within that state
        matched_facility = process.extractOne(
            city_part,
            filtered_airports["Facility Name"].dropna().unique().tolist(),
            scorer=fuzz.token_set_ratio
        )[0]

        origin_row = filtered_airports[filtered_airports["Facility Name"] == matched_facility].iloc[0]
        latitude = origin_row["latitude"]
        longitude = origin_row["longitude"]

        print("✅ Matched airport:", matched_facility)
        print("Latitude:", latitude)
        print("Longitude:", longitude)

    except Exception as e:
        print("❌ Error during matching fallback:", e)
        latitude, longitude = np.nan, np.nan
else:
    print("❌ No city/state could be extracted.")
    latitude, longitude = np.nan, np.nan


# === Extract distance + direction from summary ===
def extract_location(summary):
    match = re.search(r'([0-9]+(?:\.\d+)?)\s+(N|NE|E|SE|S|SW|W|NW)\b', summary.upper())
    return f"{match.group(1)} {match.group(2)}" if match else None

def extract_altitude(summary):
    match = re.search(r'([0-9,]+)\s+FEET', summary.upper())
    return int(match.group(1).replace(",", "")) if match else np.nan

location_cleaned = extract_location(sample_string)
altitude = extract_altitude(sample_string)
print("Extracted location:", location_cleaned)
print("Extracted altitude (ft):", altitude)

# === Bearing conversion table ===
DIRECTION_TO_BEARING = {
    "N": 0, "NNE": 22.5, "NE": 45, "ENE": 67.5, "E": 90, "ESE": 112.5, "SE": 135,
    "S": 180, "SSW": 202.5, "SW": 225, "WSW": 247.5, "W": 270, "WNW": 292.5,
    "NW": 315, "NNW": 337.5
}

# === Compute 3D location based on reference point and bearing ===
def compute_3d_coordinates(loc_cleaned, lat, lon, alt):
    try:
        dist_nm, direction = loc_cleaned.split()
        bearing = DIRECTION_TO_BEARING.get(direction.upper())
        dist_km = float(dist_nm) * 1.852
        origin = GeoPoint(lat, lon)
        dest = distance(kilometers=dist_km).destination(origin, bearing)
        return dest.latitude, dest.longitude, alt
    except Exception as e:
        print("Error computing 3D coordinates:", e)
        return np.nan, np.nan, np.nan
    

final_lat, final_lon, final_alt = compute_3d_coordinates(location_cleaned, latitude, longitude, altitude)
print("Final latitude:", final_lat)
print("Final longitude:", final_lon)
print("Final altitude (ft):", final_alt)

# === Define the 2D drone point ===
drone_point_2d = Point(final_lon, final_lat)
print("Drone point (2D):", drone_point_2d)

# === Load pre-processed airspace GeoJSON instead of parsing KMZ ===
gdf_airspace = gpd.read_file("airspaces.geojson")
print("Loaded airspace geojson with", len(gdf_airspace), "rows.")
print("gdf_airspace head:\n", gdf_airspace.head())

# === Check for intersection ===
intersecting_airspaces = gdf_airspace[gdf_airspace.geometry.contains(drone_point_2d)]
print("Intersecting airspaces:\n", intersecting_airspaces[["airspace_name", "min_altitude", "max_altitude"]])

# === Output for Pydeck visualization ===
df_intrusion = pd.DataFrame([{
    "longitude": final_lon,
    "latitude": final_lat,
    "alt_scaled": final_alt * 0.3048  # Convert ft to meters
}])
print("Pydeck dataframe:\n", df_intrusion)

sample_string: PRELIM INFO FROM FAA OPS: HARRISBURG, PA/UAS INCIDENT/1057E/CIRRUS SR20, REPORTED A COPPER UAS WHILE E BOUND AT 5,000 FEET 5 W HARRISBURG. NO EVASIVE ACTION REPORTED. PA STATE PD NOTIFIED.


/Users/vishalivallioor/anaconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


airport_codes_2 head:
       Site Id Facility Type Loc Id Effective Date Region   ADO State Id  \
0  50296.01*A       AIRPORT   18AA     2024-08-08    AAL  NONE       AK   
1  50296.02*H      HELIPORT   AA35     2024-08-08    AAL  NONE       AK   
2  50584.11*A       AIRPORT   AK46     2024-08-08    AAL  NONE       AK   
3    50009.*A       AIRPORT    ADK     2024-08-08    AAL  NONE       AK   
4  50296.03*A       AIRPORT   9AA9     2024-08-08    AAL  NONE       AK   

  State Name             County County State  ...  \
0     ALASKA             HAINES           AK  ...   
1     ALASKA             HAINES           AK  ...   
2     ALASKA  MATANUSKA-SUSITNA           AK  ...   
3     ALASKA     ALEUTIANS WEST           AK  ...   
4     ALASKA             HAINES           AK  ...   

  Airport Elevation Source Date Fuel Available Transient Storage  \
0                    2020-10-23            NaN               NaN   
1                    2014-03-13            NaN               NaN   
2  

In [7]:
matched_facility

'HARRISBURG MEDICAL CENTER'

In [3]:
latitude

NameError: name 'latitude' is not defined